In [1]:
import tkinter as tk
import cv2
import numpy as np
import pandas as pd
from tkinter import Label
from PIL import Image, ImageTk
import sys
import os
from deepface import DeepFace
import tempfile

In [2]:
class application:
    def __init__(self,root):
        self.root = root
        self.root.title("FaceAnalyst")
        self.cam1=self.camera(self.root)
        
        
    class camera:

        def analyst(self):
            cascade_path = r'C:\Users\Mehmet\anaconda3\envs\modelegitim\Lib\site-packages\data\haarcascade_frontalface_default.xml'
            face_cascade = cv2.CascadeClassifier(cascade_path)
            faces = face_cascade.detectMultiScale(self.frame, scaleFactor=1.1, minNeighbors=5)
            persons=0
            person_info=[]
            for (x_loc, y_loc, w, h) in faces:
                persons+=1
                person_info.append([persons,(str(persons)+".Kişi"),x_loc,y_loc])
                self.name_label.config(text=str(persons)+". Kişi")
                self.name_label.place(x=x_loc+500,y=y_loc)
                
                cv2.rectangle(self.frame, (x_loc, y_loc), (x_loc+w, y_loc+h), (0, 0, 0), 3)
                
                
            
            if self.back==True:
                
                self.image2_label.config(image=self.photo)
                self.image2_label.image=self.photo

                with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_file:
                    temp_image_path = temp_file.name
                    cv2.imwrite(temp_image_path, self.frame)
                
                try:
                    analysis = DeepFace.analyze(img_path=temp_image_path, actions=['age', 'gender', 'emotion'])
                    informations=""
                    for person_number in range(persons):
                        person =analysis[person_info[person_number][0]-1]
                        genders=person['gender']
                        gender=max(genders,key=genders.get)
                        gender_val=max(genders.values())
                        age=person['age'] 
                        emotions=person['emotion']
                        emotion=max(emotions,key=emotions.get)
                        emotion_val=max(emotions.values())
                    
                        informations=informations+"\n"+(
                        str(person_number+1)+". Kişinin Bilgi Tahminleri\n"+
                        "Yaş : "+str(age)+"\n"+
                        "Cinsiyet : "+str(gender)+"  %"+str(int(gender_val))+" ihtimalle\n"+
                        "Duygu Hali : "+str(emotion)+"  %"+str(int(emotion_val))+" ihtimalle")
                    self.table_label.config(text=informations)
                    
                except Exception as e:
                    self.table_label.config(text="HATA!")
                
                
                os.remove(temp_image_path)
                self.root.after(30, self.analyst)
                
            
        def open_camera(self):
            self.video=cv2.VideoCapture(0)
            self.get_img()
            self.analyst_button=tk.Button(self.root,text='Analiz',width=10,height=2,command=self.analyst)
            self.analyst_button.place(x=180,y=0)
            
        def quit(self):
            os._exit(0)
            
        def __init__(self,root):
            self.root = root

            
            
            self.image_button=tk.Button(self.root,text='Kamera aç',width=10,height=2,command=self.open_camera)
            self.quit_button=tk.Button(self.root,text='Kapat',width=10,height=2,command=self.quit)
            self.image_button.place(x=0,y=0)
            self.quit_button.place(x=90,y=0)
            self.image_label = tk.Label(self.root)
            self.image_label.place(x=0,y=50)
            self.image2_label = tk.Label(self.root)
            self.image2_label.place(x=500,y=50)
            self.name_label=tk.Label(self.root)
            self.table_label=tk.Label(self.root)
            self.table_label.place(x=1200,y=100)

           
            
        def get_img(self):
            
            self.back,self.frame=self.video.read()
            if self.back==True:
                
                self.frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
                self.image = Image.fromarray(self.frame)
                self.image=self.image.transpose(Image.FLIP_LEFT_RIGHT)
                self.photo = ImageTk.PhotoImage(image=self.image)
                
                self.image_label.config(image=self.photo)
                
            self.root.after(30, self.get_img)
            
        def __del__(self):
            if self.video.isOpened():
                self.video.release()
                self.image_label.release()

In [ ]:
root=tk.Tk()
ap1=application(root)
root.mainloop()
del root


Action: age:   0%|          | 0/3 [00:00<?, ?it/s]    